<a href="https://colab.research.google.com/github/diegoavarela/colab-tests/blob/main/Users.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations

In [ ]:
!pip3 install plotly-geo
!pip3 install plotly==5.3.1

!pip3 install geopandas==0.3.0
!pip3 install pyshp==1.2.10
!pip3 install shapely==1.6.3

# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
# Install Folium for Geographic data visualization
!pip install folium
# Install plotlyExpress
!pip install plotly_express
!pip3 install pygal_maps_world

# Imports

In [2]:
import pygal

import plotly
from plotly import __version__
print (__version__)
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from datetime import datetime
import plotly.figure_factory as ff

dic_users = {1: 'New User', 2: 'Returning User'}

# read the csv and generate a Panda
users = pd.read_csv('New vs Returning - Connected Sheet 1.csv')

5.3.1


# Filter by Date and plot New and Returning

In [ ]:
#@title Dates Range for Report
from_date = "2021-05-10" #@param {type:"date"}
to_date = "2021-09-09" #@param {type:"date"}


date_input = '%Y-%m-%d'
date_output = '%m/%d/%Y'
activity_input = '%m/%d/%Y %H:%M:%S'
# convert from
datetimeobject = datetime.strptime(from_date,date_input)
from_date = datetimeobject.strftime(date_output)

#convert to
datetimeobject = datetime.strptime(to_date,date_input)
to_date = datetimeobject.strftime(date_output)

# dates go from May-10 to Sept-9
users['date'] = '' 
for ind in users.index:
  dateactivity = datetime.strptime(users['activity'][ind],activity_input)
  users['date'][ind] = dateactivity.strftime(date_output)

# get ready to slice
users_sliced = users[(users['date'] >= from_date) & (users['date'] <= to_date)].sort_values(by='date', ascending=True)

# now we start slicing the info, we add the Amount Users field
users_sliced[ 'Amount Users'] = 0
# we only work with three columns
users_sliced = users_sliced[['user_type','client', 'Amount Users']]
# we group them
users_sliced = users_sliced.groupby([ 'user_type', 'client'])['Amount Users'].count().reset_index()
# we sort them
users_sliced = users_sliced.sort_values(by='Amount Users', ascending=False).reset_index()

# start with the plot
plt.figure(figsize=(8,10))

# bar chart 1 -> top bars -> Returning Users
bar1 = sns.barplot(x="Amount Users",  y="client", data=users_sliced, color='darkblue', orient='h')

# bottom bar ->  take only smoker=Yes values from the data
final = users_sliced[users_sliced.user_type == dic_users[1]]

# bar chart 2 -> bottom bars -> New Users - validates there are data in
try:
  bar2 = sns.barplot(x="Amount Users", y="client", data=final, estimator=sum, ci=100,  color='lightblue', orient='h')
except:
  print('bar2 is empty')

# add legend
top_bar = mpatches.Patch(color='darkblue', label='Returning Users')
bottom_bar = mpatches.Patch(color='lightblue', label='New Users')
plt.legend(handles=[top_bar, bottom_bar])

# Methods


In [21]:
    def ConvertCountryFileToCSV():

        file_input = open('countries.csv', 'r')
        file_output = open('countries_converted.csv', 'w')

        file_output.write('Code' + ',' + 'Name\n')

        for line in file_input:
            code_input = line.strip()[0:2]
            country_input = line.strip()[3:]
            file_output.write(code_input + ',' + country_input + '\n')


    def ConvertCountryFileToDict(countries):
            
            countries = dict()
            file_input = open('countries_converted.csv', 'r')

            for line in file_input:
                    line = line.strip('\n')
                    (key, val) = (line[3:],line[0:2])
                    countries[key] = val

            return countries

# Visualize Maps

In [31]:
# Fresh version of users to filter
users_sliced = users.copy()

countries = dict()
countries = ConvertCountryFileToDict(countries)

headers = ['Continent_Name','Continent_Code','Country_Name','Two_Letter_Country_Code','Three_Letter_Country_Code','Country_Number']

#Initialize map
worldmap =  pygal.maps.world.World()

# now we start slicing the info, we add the Amount Users field
users_sliced['Amount Users'] = 0
# we only work with three columns
users_sliced = users_sliced[['country','client', 'Amount Users']]

# we group them
users_sliced = users_sliced.groupby(['country'])['Amount Users'].count().reset_index()
# we sort them


#users_sliced = users_sliced.sort_values(by='Amount Users', ascending=False).reset_index()

users_sliced['Code'] = 0



my_dictionary.clear()

for ind in users_sliced.index:
    users_sliced['Code'][ind] = countries.get(users_sliced.get('country')[ind])
    my_dictionary.update({users_sliced['Code'][ind]:users_sliced['Amount Users'][ind]})

# set the title of the map
worldmap.title = 'Interactions from A+ in different Countries'

# adding the countries
worldmap.add('Countries', my_dictionary)

worldmap.render_to_png('interactions.png')




